In [1]:
import sys
import os
import pandas as pd
from pathlib import Path
import sqlite3

# Caminho da pasta "Twitch"
base_dir = Path(os.getcwd()).resolve().parent  # Ajuste se necessário

# Adicionar as pasta ao sys.path
processed_dir = base_dir / 'data' / 'processed'
sys.path.append(str(processed_dir))

In [2]:
# 1. Carregar o ficheiro CSV
csv_artist = processed_dir / 'artist_details.csv'
csv_tracks = processed_dir / 'artist_tracks.csv'
artists = pd.read_csv(csv_artist)
tracks = pd.read_csv(csv_tracks)

In [3]:
tracks["Cluster_Hierarchical"].unique()

array([2, 0, 1, 3])

In [15]:
# 2. Criar uma base de dados SQLite
spotify = sqlite3.connect('spotify.db')  # Cria o ficheiro da base de dados localmente

In [16]:
# 3. Carregar os DataFrames para tabelas separadas na base de dados
artists.to_sql('songs1', spotify, if_exists='replace', index=False)
tracks.to_sql('songs2', spotify, if_exists='replace', index=False)

14456

In [17]:
# 4. Executar queries SQL envolvendo ambas as tabelas
# Exemplo 1: Número de artistas por país
query = """
    SELECT country_born, COUNT(*) as artist_count
    FROM songs1
    GROUP BY country_born
    ORDER BY artist_count DESC
"""
result = pd.read_sql_query(query, spotify)

# Mostrar o resultado
print(result)

              country_born  artist_count
0            United States           251
1           United Kingdom            84
2                   Canada            19
3              Puerto Rico            14
4                   Sweden            12
5                Australia            12
6              Netherlands            11
7                   France             9
8                Indonesia             6
9                 Colombia             6
10                  Norway             5
11                 Germany             5
12                   Spain             3
13             New Zealand             3
14                 Jamaica             3
15                   Italy             3
16                  Brazil             3
17                 Belgium             3
18                 Romania             2
19                 Denmark             2
20               Argentina             2
21                 Senegal             1
22                  Panama             1
23              

In [18]:
# Query para contar artistas por género
query = """
    SELECT gender, COUNT(*) as count
    FROM songs1
    GROUP BY gender
    ORDER BY count DESC
"""
result = pd.read_sql_query(query, spotify)

# Mostrar o resultado
print(result)


   gender  count
0    Male    338
1  Female    130


## Spotify tracks

```sql
query = """
    SELECT t.*
    FROM spotify.hue__tmp_artist_tracks t
    JOIN spotify.hue__tmp_artist_details d
        ON t.artist_id = d.artist_id
    WHERE d.country_born = 'United States';
"""
result = pd.read_sql_query(query, spotify)

# Mostrar o resultado
print(result)
```

```sql
query = """
    SELECT t.artist_name, AVG(t.danceability) AS avg_danceability
    FROM spotify.hue__tmp_artist_tracks t
    JOIN spotify.hue__tmp_artist_details d
      ON t.artist_id = d.artist_id
    WHERE d.country_born != 'United States'
    GROUP BY t.artist_name
    ORDER BY avg_danceability DESC;
"""
result = pd.read_sql_query(query, spotify)

# Mostrar o resultado
print(result)
```

```sql
query = """
    SELECT t.album, AVG(t.energy) AS avg_energy
    FROM spotify.hue__tmp_artist_tracks t
    JOIN spotify.hue__tmp_artist_details d
      ON t.artist_id = d.artist_id
    WHERE d.country_born = 'United States'
    GROUP BY t.album
    HAVING COUNT(t.track_id) > 2
    ORDER BY avg_energy DESC;
"""
result = pd.read_sql_query(query, spotify)

# Mostrar o resultado
print(result)
```